***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [50]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-979b  GCE       4                                       RUNNING  us-central1-c


# Imports & Setup

In [51]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [52]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 12 08:37 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [54]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [55]:
spark

In [56]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'elad_318640828_anchor_bucket' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name, prefix='Parquet_Files')
for b in blobs:
    if b.name.endswith('.parquet'):
        paths.append(full_path+b.name)
# print(paths)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [57]:
parquetFile = spark.read.parquet(*paths)
doc_anchor_pairs_pre = parquetFile.select("anchor_text","id").rdd
doc_titles_pairs = parquetFile.select("id","title").rdd

In [58]:
def mapping_as_should(doc_anchor_pair):
    edge_set = []
    for id, anchor_text in doc_anchor_pair[0]:
        if (id, anchor_text) not in edge_set:
            edge_set.append((id, anchor_text))
    return edge_set

doc_anchor_pairs = doc_anchor_pairs_pre.flatMap(mapping_as_should).distinct().groupByKey()
doc_anchor_pairs = doc_anchor_pairs.mapValues(list).map(lambda x: (" ".join(x[1]), x[0]))

In [59]:
# doc_anchor_pairs.take(1)

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [60]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [61]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

23/01/12 18:34:33 WARN org.apache.spark.SparkContext: The path /home/dataproc/inverted_index_gcp.py has been added already. Overwriting of added paths is not supported in the current version.


In [62]:
from inverted_index_gcp import InvertedIndex

In [63]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124


def token2bucket_id(token):
    return int(_hash(token), 16) % NUM_BUCKETS


# PLACE YOUR CODE HERE
def word_count(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in
    `all_stopwords` and return entries that will go into our posting lists.
    Parameters:
    -----------
      text: str
        Text of one document
      id: int
        Document id
    Returns:
    --------
      List of tuples
        A list of (token, (doc_id, tf)) pairs
        for example: [("Anarchism", (12, 5)), ...]
    '''
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]

    final_tokens = []
    for tok in tokens:
        if tok not in all_stopwords:
            final_tokens.append(tok)
    tf_dict = Counter(final_tokens)
    returning = []
    for elem in tf_dict.items():
        returning.append((elem[0], (id, elem[1])))
    return returning

######################
def document_length(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in
    `all_stopwords` and return entries that will go into our posting lists.
    Parameters:
    -----------
      text: str
        Text of one document
      id: int
        Document id
    Returns:
    --------
      List of tuples
        A list of (token, (doc_id, tf)) pairs
        for example: [("Anarchism", (12, 5)), ...]
    '''
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]

    final_tokens = []
    for tok in tokens:
        if tok not in all_stopwords:
            final_tokens.append(tok)
    res = []
    res.append((len(final_tokens), id))

    return res

##################
def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
      unsorted_pl: list of tuples
        A list of (wiki_id, tf) tuples
    Returns:
    --------
      list of tuples
        A sorted posting list.
    '''
    finale = []
    finale = sorted(unsorted_pl, key= lambda x: x[1], reverse=True)
    return finale

##################
def sorting_to_DL(row):
    a, b_list = row
    b = b_list[0]
    return (a, b)

##################
def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
    Parameters:
    -----------
      postings: RDD
        An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each element is a (token, df) pair.
    '''
    finale = []
    finale = postings.map(lambda x: (x[0], len(x[1])))

    return finale


##################
def partition_postings_and_write(postings):
    ''' A function that partitions the posting lists into buckets, writes out
    all posting lists in a bucket to disk, and returns the posting locations for
    each bucket. Partitioning should be done through the use of `token2bucket`
    above. Writing to disk should use the function  `write_a_posting_list`, a
    static method implemented in inverted_index_colab.py under the InvertedIndex
    class.
    Parameters:
    -----------
      postings: RDD
        An RDD where each item is a (w, posting_list) pair.
    Returns:
    --------
      RDD
        An RDD where each item is a posting locations dictionary for a bucket. The
        posting locations maintain a list for each word of file locations and
        offsets its posting list was written to. See `write_a_posting_list` for
        more details.
    '''

    file_location_dictionary = {}
    lama = postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey().map(
        lambda x: InvertedIndex.write_a_posting_list(x, bucket_name))

    return lama

In [64]:
# time the index creation time
t_start = time()
# word counts map
word_counts_text = doc_anchor_pairs.flatMap(lambda x: word_count(x[0], x[1]))
doc_len_dict = doc_anchor_pairs.flatMap(lambda x: document_length(x[0], x[1]))

postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
# postings_text_doc_len_dict = doc_len_dict.groupByKey().mapValues(reduce_word_counts).map(sorting_to_DL)
# to_DL = postings_text_doc_len_dict.collectAsMap()
# filtering postings and calculate df
postings_filtered_text = postings_text.filter(lambda x: len(x[1])>50)

w2df_text = calculate_df(postings_filtered_text)

w2df_dict_text = w2df_text.collectAsMap()

# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered_text).collect()

index_const_time = time() - t_start

# test index construction time
print(index_const_time)

625.0677099227905


In [40]:
# # time the index creation time
# t_start = time()

# words = doc_anchor_pairs.flatMap(lambda y: map(lambda x: word_count(x[1],(y[1],x[0])),y[0]))
# word_counts=words.flatMap(lambda x:x)
# wordsone=word_counts.mapValues(lambda y:(y[0][1],y[1]))

# postings=wordsone.groupByKey().mapValues(lambda x:union_count(sorted(list(x),key=lambda x:x[0])))
# postings_filtered = postings.filter(lambda x: len(x[1])>50)
# w2df = calculate_df(postings_filtered)
# w2df_dict = w2df.collectAsMap()
# posting_locs_list = partition_postings_and_write(postings_filtered).collect()

# index_const_time = time() - t_start

# # test index construction time
# print(index_const_time)

In [65]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

In [66]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict_text
# Add dictionary that map document id to his title
inverted.id2titles = doc_titles_pairs.collectAsMap()
# Add DL dictionary
# inverted.DL = to_DL
# write the global stats out
inverted.write_index('.', 'index_anchor')
# upload to gs
index_src = "index_anchor.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index_anchor.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1 files][170.2 MiB/170.2 MiB]                                                
Operation completed over 1 objects/170.2 MiB.                                    


In [67]:
inverted.df["king"]

57297

Putting it all together

In [44]:
# # Create inverted index instance
# inverted = InvertedIndex()
# # Adding the posting locations dictionary to the inverted index
# inverted.posting_locs = super_posting_locs
# # Add the token - df dictionary to the inverted index
# inverted.df = w2df_dict
# totals=postings.map(lambda x:calculate_total_term(x[0],x[1]))
# inverted.term_total=totals.collectAsMap()
# # write the global stats out
# inverted.write_index('.', 'index_anchor')
# # upload to gs
# index_src = "index_anchor.pkl"
# index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
# !gsutil cp $index_src $index_dst

In [68]:
!gsutil ls -lh $index_dst

170.16 MiB  2023-01-12T18:45:36Z  gs://elad_318640828_anchor_bucket/postings_gcp/index_anchor.pkl
TOTAL: 1 objects, 178429572 bytes (170.16 MiB)


In [69]:
from itertools import islice

def take(n, iterable):
    """Return the first n items of the iterable as a list."""
    return list(islice(iterable, n))

n = 100
n_items = take(n, inverted.id2titles.items())



print(n_items)

[(4045403, 'Foster Air Force Base'), (4045413, 'Torino Palavela'), (4045419, 'Mad About the Boy'), (4045426, 'Shayne Breuer'), (4045432, 'Parantaka I'), (4045456, 'Arundel (UK Parliament constituency)'), (4045466, 'Andrew Martinez'), (4045471, 'Vancouver VooDoo'), (4045479, 'Invisible plane'), (4045516, 'Shopping channel'), (4045519, 'Turgay (river)'), (4045523, 'Turgay'), (4045525, 'Heinrich Johann Nepomuk von Crantz'), (4045532, 'KUT'), (4045544, 'Dodge Cove'), (4045546, 'Triphone'), (4045554, 'Government House (Nova Scotia)'), (4045575, 'Bud Abell'), (4045577, 'Susette La Flesche'), (4045581, 'Pomme de Terre River (Missouri)'), (4045584, 'Elizabethan Club'), (4045604, 'Ray Abruzzese'), (4045607, 'Holy Family High School (Broomfield, Colorado)'), (4045640, 'Deutsche Gesellschaft für Internationale Zusammenarbeit'), (4045651, 'Ken Adamson'), (4045659, 'Sir William Mackinnon, 1st Baronet'), (4045661, 'List of highest-income ZIP Code Tabulation Areas in the United States'), (4045672, '2

In [47]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 10
print(f'I used {cost} USD credit during the course of this assignment')

I used 10 USD credit during the course of this assignment


In [24]:
def generate_graph(pages):
''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
    edges = pages.flatMap(lambda x: set([(x[0], link[0]) for link in x[1]]))
    list_vertices = edges.map(lambda x: ((x[1], 0)))
    groupi = edges.groupByKey()
    keys_vertices = groupi.map(lambda x: (x[0], 0))
    vertices = (keys_vertices + list_vertices).groupByKey().map(lambda x: [x[0]])

    return edges, vertices

IndentationError: expected an indented block (2690464089.py, line 17)

In [ ]:
# test graph construction efficiency (10 points)
t_start = time()
# construct the graph for a small sample of (1000) pages
edges, vertices = generate_graph(pages_links)
# insert this parameters to our inverted index
inverted.vertices = vertices
inverted.edges = edges
# time the actual execution
v_cnt, e_cnt = vertices.count(), edges.count()
graph_const_time = time() - t_start
# test that the implementation is efficient (<20 seconds)
print(graph_const_time)

In [ ]:
edgesDF = edges.toDF(['src', 'dst']).repartition(4, 'src')
verticesDF = vertices.toDF(['id']).repartition(4, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=10)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv('pr', compression="gzip")
pr.show()